# How to use the package

Using the genetic algorithms package is fairly easy. The user only has to specify 2 functions:

    1. desiredState() which returns an array representing the full wavefunction of the desired state.
    2. evuluateInd(individual) which takes in an individual and returns a tuple representing the fitness of the individual, for example, (error, length).

Now let's start by importing the necessary modules.

In [ ]:
import projectq
from projectq.ops import H, X, Y, Z, T, Tdagger, S, Sdagger, CNOT, Measure, All, Rx, Ry, Rz, SqrtX
import numpy as np
import copy
from constants import *
from deap import creator, base, tools
from candidate import Candidate
from constants import *
from evolution import crossoverInd, mutateInd, selectAndEvolve, geneticAlgorithm

Now that we have all the modules we need, we can define the desired state function. Let's say we are using 2 qubits, so the wavefunction has 4 

In [ ]:
def desiredState():
    wf = [0.3,0.5,0.66,0]
    return wf

Let's say we are trying to minimize the error and the circuit length at the same time. So in that case our evaluateInd function will be something like:

In [ ]:
def evaluateInd(individual):
  '''
  This function should take an individual,possibly an instance of Candidate class, 
  and return a tuple where each element of the tuple is an objective.
  An example objective would be (error,circuitLen) where:
    error = |1 - < createdState | wantedState > 
    circuitLen = len(individual.circuit) / MAX_CIRCUIT_LENGTH
    MAX_CIRCUIT_LENGTH is the expected circuit length for the problem. 
  '''
  wanted = desiredState()
  got = individual.simulateCircuit()
  error = 1 - np.absolute(np.vdot(wanted,got))
  if len(individual.circuit)>0 and len(individual.circuit)<MAX_CIRCUIT_LENGTH:
    return (error, len(individual.circuit)/MAX_CIRCUIT_LENGTH)
  else:
    # If this is the case, then the circuit is longer that what we want.
    return (error,1.0)

Finally, we define our main function where we define number of qubits, allowed gates, etc. 

In [ ]:
'''
You should initialize:
numberOfQubits : number of qubits to be used for the problem
allowedGates   : allowed set of gates. Default is [Rz,SX,X,CX]
problemName    : output of the problem will be stored at ./outputs/problemName.txt
problemDescription : A small header describing the problem.
fitnessWeights : A tuple describing the weight of each objective. A negative
    weight means that objective should be minimized, a positive weight means
    that objective should be maximized. For example, if you want to represent 
    your weights as (error,circuitLen) and want to minimize both with equal 
    weight you can just define fitnessWeights = (-1.0,-1.0). Only the relative 
    values of the weights have meaning. BEWARE that they are multiplied and summed 
    up while calculating the total fitness, so you might want to normalize them.
'''
# Initialize your variables
numberOfQubits = 2
# Let's try to use the basis gate of IBM Quantum Computers
allowedGates = [Rx,Ry,Rz,X,CNOT] 
problemName = "kindaarbitrary"
problemDescription = "Kind of Arbitrary State initalization for:\n"
#problemDescription += str(c0)+"|00>"+str(c1)+"|01>"+str(c2)+"|10>"+str(c3)+"|11>\n"
problemDescription += "numberOfQubits="+str(numberOfQubits)+"\n"
problemDescription += "allowedGates="+str(allowedGates)+"\n"
# trying to minimize error and length !
fitnessWeights = (-1.0, -1.0)

# Create the type of the individual
creator.create("FitnessMin", base.Fitness, weights=fitnessWeights)
creator.create("Individual", Candidate, fitness=creator.FitnessMin)
# Initialize your toolbox and population
toolbox = base.Toolbox()
toolbox.register("individual", creator.Individual,numberOfQubits,allowedGates)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# Register the necessary functions to the toolbox
toolbox.register("mate", crossoverInd, toolbox=toolbox)
toolbox.register("mutate", mutateInd)
toolbox.register("select", tools.selNSGA2)
toolbox.register("selectAndEvolve", selectAndEvolve)
toolbox.register("evaluate", evaluateInd)

# Get it running
NGEN = 100       # For how many generations should the algorithm run ? 
POPSIZE = 500  # How many individuals should be in the population ? 
verbose = False # Do you want functions to print out information. 
                # Note that they will print out a lot of things. 

# Initialize a random population
pop = toolbox.population(n=POPSIZE)
# Run the genetic algorithm
pop = geneticAlgorithm(pop, toolbox, NGEN, problemName, problemDescription, epsilon)


In [ ]:
pop.sort(key=lambda x: x.fitness.values[0])

In [ ]:
pop[0].fitness.values

# 3 Qubit Example

Now, let us do the same thing for a 3-qubit case. Let's say we are trying to initialize the state:  

$$\Psi = [0.1,\ 0.4,\ 0.3,\ 0.3,\ 0.6,\ 0.4,\ 0.3,\ 0.2]^T$$

We allow gates the Rx,Ry,Rz,CNOT gates and the qubit connectivity is linear. That is, 0 <-> 1 <-> 2.

In [ ]:
def desiredState():
  wf = [0.1,0.4,0.3,0.3,0.6,0.4,0.3,0.2]
  return wf 

In [ ]:
def evaluateInd(individual):
  '''
  This function should take an individual,possibly an instance of Candidate class, 
  and return a tuple where each element of the tuple is an objective.
  An example objective would be (error,circuitLen) where:
    error = |1 - < createdState | wantedState > 
    circuitLen = len(individual.circuit) / MAX_CIRCUIT_LENGTH
    MAX_CIRCUIT_LENGTH is the expected circuit length for the problem. 
  '''
  wanted = desiredState()
  got = individual.simulateCircuit()
  error = 1 - np.absolute(np.vdot(wanted,got))
  if len(individual.circuit)>0 and len(individual.circuit)<MAX_CIRCUIT_LENGTH:
    return (error, len(individual.circuit)/MAX_CIRCUIT_LENGTH)
  else:
    # If this is the case, then we shorten the circuit.
    individual.circuit = individual.circuit[:MAX_CIRCUIT_LENGTH]
    return (error,1.0)

In [ ]:
'''
You should initialize:
numberOfQubits : number of qubits to be used for the problem
allowedGates   : allowed set of gates. Default is [Rz,SX,X,CX]
problemName    : output of the problem will be stored at ./outputs/problemName.txt
problemDescription : A small header describing the problem.
fitnessWeights : A tuple describing the weight of each objective. A negative
    weight means that objective should be minimized, a positive weight means
    that objective should be maximized. For example, if you want to represent 
    your weights as (error,circuitLen) and want to minimize both with equal 
    weight you can just define fitnessWeights = (-1.0,-1.0). Only the relative 
    values of the weights have meaning. BEWARE that they are multiplied and summed 
    up while calculating the total fitness, so you might want to normalize them.
'''
# Initialize your variables
numberOfQubits = 3
# Let's try to use the basis gate of IBM Quantum Computers
allowedGates = [Ry,Rx,Rz,CNOT] 
connectivity = [(0,1),(1,0),(1,2),(2,1)]
problemName = "kindaarbitrary3qubit"
problemDescription = "Kind of Arbitrary State initalization for:\n"
problemDescription += "numberOfQubits="+str(numberOfQubits)+"\n"
problemDescription += "allowedGates="+str(allowedGates)+"\n"
# trying to minimize error and length !
fitnessWeights = (-1.0, -1.0)

# Create the type of the individual
creator.create("FitnessMin", base.Fitness, weights=fitnessWeights)
creator.create("Individual", Candidate, fitness=creator.FitnessMin)
# Initialize your toolbox and population
toolbox = base.Toolbox()
toolbox.register("individual", creator.Individual,numberOfQubits,allowedGates,connectivity)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# Register the necessary functions to the toolbox
toolbox.register("mate", crossoverInd, toolbox=toolbox)
toolbox.register("mutate", mutateInd)
toolbox.register("select", tools.selNSGA2)
toolbox.register("selectAndEvolve", selectAndEvolve)
toolbox.register("evaluate", evaluateInd)

# Get it running
NGEN = 100       # For how many generations should the algorithm run ? 
POPSIZE = 500  # How many individuals should be in the population ? 
verbose = False # Do you want functions to print out information. 
                # Note that they will print out a lot of things. 

# Initialize a random population
pop = toolbox.population(n=POPSIZE)
# Run the genetic algorithm
pop = geneticAlgorithm(pop, toolbox, NGEN, problemName, problemDescription, epsilon)


In [ ]:
pop.sort(key=lambda x: x.fitness.values[0])

In [ ]:
pop[0].drawCircuit()


In [ ]:
pop[0].fitness.values

# Features to be added
Although the package works as it is, there are some important features that needs to be added:

    - Add Swap Gate
    - Add SqrtX Gate to the package
    - Evaluate the cost of a circuit
    - Two-level Mutation
    - Combining Gates
    - Qubit Mapping Feature